## Découverte ML 

### Exercice Titanic (Vidéo Machine Learnia)

In [ ]:
import pandas as pd 
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
titanic = sns.load_dataset('titanic')
titanic.shape

titanic.describe()

In [17]:
titanic = titanic[['survived','pclass','sex','age']]
titanic.dropna(axis=0, inplace=True)
titanic['sex'].replace(['male', 'female'],[0,1],inplace=True)
titanic.head

<bound method NDFrame.head of      survived  pclass  sex   age
0           0       3    0  22.0
1           1       1    1  38.0
2           1       3    1  26.0
3           1       1    1  35.0
4           0       3    0  35.0
..        ...     ...  ...   ...
885         0       3    1  39.0
886         0       2    0  27.0
887         1       1    1  19.0
889         1       1    0  26.0
890         0       3    0  32.0

[714 rows x 4 columns]>

In [18]:
from sklearn.neighbors import KNeighborsClassifier

In [19]:
model = KNeighborsClassifier(n_neighbors=6)

In [20]:
y = titanic['survived']
x = titanic.drop('survived',axis=1)

In [21]:
model.fit(x,y)
model.score(x,y)

0.8221288515406162

In [23]:
def survie(model, pclass=3, sex=1, age=22):
    x=np.array([pclass, sex, age]).reshape(1,3)
    print(model.predict(x))
    print(model.predict_proba(x))

In [24]:
survie(model)

[1]
[[0.33333333 0.66666667]]


C:\Users\lucie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\lucie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


### Model Selection (Vidéo Machine Learnia)

In [25]:
model = KNeighborsClassifier(n_neighbors=6)

model.fit(X_train, y_train)
print('Train score:', model.score(X_train, y_train))
print('Test score:', model.score(X_test, y_test))

NameError: name 'X_train' is not defined

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val_score(KNeighborsClassifier(3), X_train, y_train, cv=5,metric='accuracy').mean()

In [ ]:
val_score = []
for k in range(1, 50):
    score=cross_val_score(KNeighborsClassifier(k), X_train, y_train, cv=5).mean()
    val_score.append(score)

plt.plot(val_score)

In [ ]:
from sklearn.model_selection import validation_curve

In [ ]:
model = KNeighborsClassifier()
k=np.arrange(1,50)

train_score, val_score = validation_curve(model,X_train,y_train, 'n_neighbors',k,cv=5)

In [ ]:
val_score.shape

In [ ]:
val_score.mean(axis=1)
plt.plot(k, val_score.mean(axis=1))
plt.plot(k, train_score.mean(axis=1),label='train')

plt.ylabel('score')
plt.xlabel('n_neighbors')
plt.legend()

### GridSearchCV (Vidéo Machine Learnia)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid={'n_neighbors' : np.arrange(1,20), 'metric':['euclidean','manhattan']}
grid=GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid.fit(x_train,y_train)

In [ ]:
grid.best_score_
grid.best_params_
model=grid.best_estimator_
model.score(X_test,y_test)


In [ ]:
from sklearn.matrics import confusion_matrix
confusion_matrix(y_test, model.predict(X_test))

### LearningCurve (Vidéo Machine Learnia)

In [ ]:
from sklearn.model_selection import learning_curve
N, train_score, val_score = learning_curve(model, X_train, y_train, train_sizes = np.linspace(0.2, 1.0,5))

plt.plot(N,train_score.mean(axis=1),label='train')
plt.plot(N, val8score.mean(axis=1), label='validation')
plt.xlabel('train_sizes')
plt.legend()

# Partie Exercice en Autonomie

### House Pricing

##### Première approche naïve

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

In [107]:
data = pd.read_csv('trainHouse.csv')

In [108]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
data.info()

In [110]:
X = data[['PoolArea','GarageArea','KitchenAbvGr','BedroomAbvGr','HalfBath','FullBath','2ndFlrSF','1stFlrSF','YearBuilt','LotArea']]
y = data['SalePrice']
n_samples=len(X)

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8)

In [112]:
model = DecisionTreeRegressor(random_state=1)

In [113]:
model.fit(X_train, y_train)

DecisionTreeRegressor(random_state=1)

In [114]:
y_pred = model.predict(X_test)

In [116]:
mse = mean_absolute_error(y_test, y_pred)

print(f'Mean Absolute Error: {mse}')

Mean Absolute Error: 31572.479452054795


##### Seconde approche avec plus de critères et le choix d'hyper paramètres pour DecisionTreeRegressor

In [117]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

In [118]:
dataTest = pd.read_csv('testHouse.csv')
dataTrain = pd.read_csv('trainHouse.csv')

In [119]:
X = data[['PoolArea','GarageArea','KitchenAbvGr','BedroomAbvGr','HalfBath','FullBath','2ndFlrSF','1stFlrSF','YearBuilt','LotArea']]
y = data['SalePrice']
n_samples=len(X)

In [120]:
model = DecisionTreeRegressor(random_state=1)

In [125]:
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}
grid=GridSearchCV(DecisionTreeRegressor(), param_grid)
grid.fit(X_train,y_train)

C:\Users\lucie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
180 fits failed out of a total of 540.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\lucie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\lucie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "C:\Users\lucie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base

GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [None, 10, 20, 30],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]})

In [126]:
grid.best_score_
grid.best_params_
model=grid.best_estimator_
model.fit(X_train, y_train)
model.score(X_test,y_test)

0.6080798370511608

In [129]:
y_pred = model.predict(X_test)

In [130]:
mse = mean_absolute_error(y_test, y_pred)

print(f'Mean Absolute Error: {mse}')

Mean Absolute Error: 26322.474913824157


##### Seconde approche avec plus de critères et le choix d'hyper paramètres pour RandomForestRegressor

In [136]:
model = RandomForestRegressor(n_estimators=100, max_depth=None, random_state=42)

In [ ]:
X = data[['PoolArea','GarageArea','KitchenAbvGr','BedroomAbvGr','HalfBath','FullBath','2ndFlrSF','1stFlrSF','YearBuilt','LotArea']]
y = data['SalePrice']
n_samples=len(X)

In [134]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8)

In [137]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}
grid=GridSearchCV(RandomForestRegressor(), param_grid)
grid.fit(X_train,y_train)

C:\Users\lucie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
540 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\lucie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\lucie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "C:\Users\lucie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\bas

GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_depth': [None, 10, 20, 30],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]})

In [138]:
grid.best_score_
grid.best_params_
model=grid.best_estimator_
model.fit(X_train, y_train)
model.score(X_test,y_test)

0.8224936436390422

In [139]:
y_pred = model.predict(X_test)

In [141]:
mse = mean_absolute_error(y_test, y_pred)
print(f'Mean absolute Error: {mse}')

Mean absolute Error: 19655.37479535276


##### Seconde approche avec plus de critères et le choix d'hyper paramètres pour GradientBoostingRegressor

In [142]:
from sklearn.ensemble import GradientBoostingRegressor

In [143]:
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

In [144]:
X = data[['PoolArea','GarageArea','KitchenAbvGr','BedroomAbvGr','HalfBath','FullBath','2ndFlrSF','1stFlrSF','YearBuilt','LotArea']]
y = data['SalePrice']
n_samples=len(X)

In [145]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8)

In [146]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 0.9, 1.0]
}
grid=GridSearchCV(GradientBoostingRegressor(), param_grid)
grid.fit(X_train,y_train)

GridSearchCV(estimator=GradientBoostingRegressor(),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 4, 5], 'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200],
                         'subsample': [0.8, 0.9, 1.0]})

In [147]:
grid.best_score_
grid.best_params_
model=grid.best_estimator_
model.fit(X_train, y_train)
model.score(X_test,y_test)

0.7496102685261112

In [148]:
y_pred = model.predict(X_test)

In [149]:
mse = mean_absolute_error(y_test, y_pred)
print(f'Mean absolute Error: {mse}')

Mean absolute Error: 21205.22993266918
